In [1]:
import numpy as np
import pandas as pd
from keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

E0000 00:00:1733162010.744287      13 common_lib.cc:798] Could not set metric server port: INVALID_ARGUMENT: Could not find SliceBuilder port 8471 in any of the 0 ports provided in `tpu_process_addresses`="local"
=== Source Location Trace: ===
learning/45eac/tfrc/runtime/common_lib.cc:479
D1202 17:53:30.752612632      13 config.cc:196]                        gRPC EXPERIMENT call_status_override_on_cancellation   OFF (default:OFF)
D1202 17:53:30.752629064      13 config.cc:196]                        gRPC EXPERIMENT call_v3                                OFF (default:OFF)
D1202 17:53:30.752632893      13 config.cc:196]                        gRPC EXPERIMENT canary_client_privacy                  ON  (default:ON)
D1202 17:53:30.752635730      13 config.cc:196]                        gRPC EXPERIMENT capture_base_context                   ON  (default:ON)
D1202 17:53:30.752638458      13 config.cc:196]                        gRPC EXPERIMENT client_idleness                        ON  (defau

In [2]:
!pip install nibabel

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 32.8 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [3]:
import nibabel as nib
import tensorflow as tf

In [4]:
df="/kaggle/input/liver-tumor-segmentation"

In [5]:
# train_data=image_dataset_from_directory(
#     df,
#     image_size=(250,250),
#     batch_size=32,
#     seed=123,
#     shuffle=True,
#     validation_split=0.2,
#     subset="training"
# )
# val_data=image_dataset_from_directory(
#     df,
#     image_size=(250,250),
#     batch_size=32,
#     seed=123,
#     shuffle=True,
#     validation_split=0.2,
#     subset="validation"
# )

In [7]:
df2="/kaggle/input/liver-tumor-segmentation/segmentations"
new_df=[]
from skimage.transform import resize
for file in os.listdir(df2):
    # for file in filename:
        
        data=os.path.join(df2,file)
        if os.path.isfile(data):
            da1=nib.load(data)
            da=da1.get_fdata()
            da_resized = resize(da, (512, 512, 256), anti_aliasing=True)

            new_df.append(da_resized)

        # new_df.append(da)
new_df=np.array(new_df)

In [8]:
new_df.shape

(131, 512, 512, 256)

In [9]:
# import matplotlib.pyplot as plt
# plt.imshow(new_df[0])
# plt.title("Middle Slice")
# plt.show()

In [10]:
label_shape = np.random.randint(0,2,(131,64, 64, 32, 1)) 
label_shape.shape

(131, 64, 64, 32, 1)

# # # cnn# 

In [11]:
from keras.layers import Layer,Dense,Conv3D,MaxPooling3D,Flatten,Input,Dropout,LeakyReLU
from keras.models import Model,Sequential
from keras.initializers import GlorotUniform
from keras.regularizers import l2
from keras.callbacks import EarlyStopping
import tensorflow as tf

In [12]:
data=np.expand_dims(new_df,axis=-1)
data.shape

(131, 512, 512, 256, 1)

In [13]:
inputs=Input(shape=(512, 512, 256,1))
C=Conv3D(32,(3,3,3),padding="same",activation="relu",kernel_initializer=GlorotUniform,kernel_regularizer=l2(0.01))(inputs)
C=Conv3D(32,(3,3,3),padding="same",activation="relu",kernel_initializer=GlorotUniform,kernel_regularizer=l2(0.01))(C)
C=MaxPooling3D(pool_size=(2,2,2),strides=(2,2,2))(C)

C=Conv3D(64,(3,3,3),padding="same",activation="relu",kernel_initializer=GlorotUniform,kernel_regularizer=l2(0.01))(C)
C=Conv3D(64,(3,3,3),padding="same",activation="relu",kernel_initializer=GlorotUniform,kernel_regularizer=l2(0.01))(C)
C=MaxPooling3D(pool_size=(2,2,2),strides=(2,2,2))(C)

C=Conv3D(128,(3,3,3),padding="same",activation="relu",kernel_initializer=GlorotUniform,kernel_regularizer=l2(0.01))(C)
C=Conv3D(128,(3,3,3),padding="same",activation="relu",kernel_initializer=GlorotUniform,kernel_regularizer=l2(0.01))(C)
C=MaxPooling3D(pool_size=(2,2,2),strides=(2,2,2))(C)
# D=Flatten()(C)
# D=Dense(32,activation="relu")(D)
# outputs=Dense(1,activation="sigmoid")(D)
outputs=Conv3D(1,(3,3,3),activation="sigmoid",padding="same")(C)
model=Model(inputs,outputs)
model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])
model.summary()

I0000 00:00:1733163091.940356      13 service.cc:145] XLA service 0x568a88585440 initialized for platform TPU (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1733163091.940417      13 service.cc:153]   StreamExecutor device (0): TPU, 2a886c8
I0000 00:00:1733163091.940421      13 service.cc:153]   StreamExecutor device (1): TPU, 2a886c8
I0000 00:00:1733163091.940425      13 service.cc:153]   StreamExecutor device (2): TPU, 2a886c8
I0000 00:00:1733163091.940428      13 service.cc:153]   StreamExecutor device (3): TPU, 2a886c8
I0000 00:00:1733163091.940431      13 service.cc:153]   StreamExecutor device (4): TPU, 2a886c8
I0000 00:00:1733163091.940440      13 service.cc:153]   StreamExecutor device (5): TPU, 2a886c8
I0000 00:00:1733163091.940442      13 service.cc:153]   StreamExecutor device (6): TPU, 2a886c8
I0000 00:00:1733163091.940445      13 service.cc:153]   StreamExecutor device (7): TPU, 2a886c8


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 512, 512, 256,  │             0 │
│                                 │ 1)                     │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d (Conv3D)                 │ (None, 512, 512, 256,  │           896 │
│                                 │ 32)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_1 (Conv3D)               │ (None, 512, 512, 256,  │        27,680 │
│                                 │ 32)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d (MaxPooling3D)    │ (None, 256, 256, 128,  │             0 │
│                                 │ 32)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_2 (Conv3D)               │ (None, 256, 256, 128,  │        55,360 │
│                                 │ 64)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_3 (Conv3D)               │ (None, 256, 256, 128,  │       110,656 │
│                                 │ 64)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d_1 (MaxPooling3D)  │ (None, 128, 128, 64,   │             0 │
│                                 │ 64)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_4 (Conv3D)               │ (None, 128, 128, 64,   │       221,312 │
│                                 │ 128)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_5 (Conv3D)               │ (None, 128, 128, 64,   │       442,496 │
│                                 │ 128)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d_2 (MaxPooling3D)  │ (None, 64, 64, 32,     │             0 │
│                                 │ 128)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_6 (Conv3D)               │ (None, 64, 64, 32, 1)  │         3,457 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 861,857 (3.29 MB)

 Trainable params: 861,857 (3.29 MB)

 Non-trainable params: 0 (0.00 B)

In [14]:
# labels = np.random.randint(0, 2,(512, 512, 75, 1))  # Binary labels
# label.shape
model.output_shape

(None, 64, 64, 32, 1)

In [15]:
data[:12].shape

(12, 512, 512, 256, 1)

In [16]:
# import numpy as np

# data = np.expand_dims(data, axis=-1)  # Adds channel dimension
# data = np.expand_dims(data, axis=2)  # Adds the second spatial dimension
# print(data.shape)  # Should output: (32, 512, 512, 75, 1)


In [17]:
data.shape

(131, 512, 512, 256, 1)

In [18]:
model.fit(data[:12],label_shape[:12])

NotFoundError: Graph execution error:

Detected at node StatefulPartitionedCall defined at (most recent call last):
  File "/usr/local/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/local/lib/python3.10/runpy.py", line 86, in _run_code

  File "/usr/local/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/usr/local/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/usr/local/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/usr/local/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/usr/local/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/local/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/local/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/usr/local/lib/python3.10/site-packages/ipykernel/kernelbase.py", line 545, in dispatch_qu

In [ ]:
# data = tf.random.normal((32,512, 512, 75, 1))  # Input data
# labels = np.random.randint(0, 2, (32, 1))  # Binary labels

# model.fit(data, labels, batch_size=32, epochs=10)


# # Quantum deep learning# 

In [19]:
!pip install cirq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.5/60.5 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 532.7/532.7 kB 6.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.3/69.3 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 596.5/596.5 kB 8.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 12.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.8/202.8 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 25.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 36.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.6/45.6 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [20]:
import cirq


In [25]:
def quantum_circuit(data):
    qubit=[cirq.GridQubit(i,j) for i in range(12) for j in range(12)]
    circuit=cirq.Circuit()
    for i in range(12):
        for j in range(12):
            index=i*12+j
            circuit.append(cirq.rx(data[index]*np.pi).on(qubit[index]))
            circuit.append(cirq.measure(qubit[index],f=f"qubit_{i}_{j}"))
    return circuit,qubit
def run_quantum_circuit(data):
    results=[]
    simulator=cirq.Simulator()
    for da in data:
        circuit,qubit=quantum_circuit(da)
        result=simulator.run(circuit)
        measure0=[]
        for i in range(len(da)):
            measure0.append(result.measurement[f"qubit_{i}_{j}"][0,0])
            results.append(measure0)
    return np.array(results,dtype=np.float32)
class QuantumLayer(Layer):
    def __init__(self):
        super(QuantumLayer,self).__init__()
    def call(self,input):
        
        outputs=tf.numpy_function(run_quantum_circuit,[input],tf.float32)
        outputs.set_shape((input.shape[0],12*12))
        return outputs
model2=Sequential([
    QuantumLayer(),
    Conv3D(1,(3,3,3),activation="sigmoid")
])
model2.compile(optimizer="adam",loss="binary_crossentropy",metrics=['accuracy'])
model2.fit(data[:12],label_shape[:12])

ValueError: Input 0 of layer "conv3d_7" is incompatible with the layer: expected min_ndim=5, found ndim=2. Full shape received: (None, 144)

In [ ]:
# model.fit